# Подготовительные действия

In [ ]:
import requests
import pandas as pd
from utils import *

In [ ]:
t = Timer()

In [ ]:
API_KEY = input()
HEADERS = {'accept': 'application/json',
           'X-API-KEY': API_KEY}

In [ ]:
REVIEWS_ROUTE = 'https://kinopoiskapiunofficial.tech/api/v1/reviews'

Получение респонса с рецензиями фильма

In [ ]:
def get_reviews(film_id, page):
    return requests.get(REVIEWS_ROUTE,
                        dict(filmId=film_id,
                             page=page),
                        headers=HEADERS)

Рецензии, полученные с помощью API, содержат некоторые служебные символы, которые необходимо убрать для дальнейшей работы

In [ ]:
def clear_text(text):
    return text.replace('\n', '')\
        .replace('\r', '')\
        .replace('\t', '')

Поскольку API разрешает делать только 20 запросов в секунду, необходимо каждые 19 запросов ждать одну секунду

In [ ]:
def check_wait(count):
    count += 1
    if count % 19 == 0:
        time.sleep(1)
    return count

Во время извлечения отзывов можно показывать прогресс выполнения, а также статистику по уже полученным рецензиям, чтобы видеть, сколько из них  положительных, отрицательных и нейтральных

In [ ]:
def update_output(pos, neg, neu, timer, i, total_i):
    clear_output(True)
    print(progress_bar(i, total_i, size=23))
    print()
    print(f'+------------+----------+')
    print(f'| + POSITIVE | {pos:>8} |')
    print(f'+------------+----------+')
    print(f'| - NEGATIVE | {neg:>8} |')
    print(f'+------------+----------+')
    print(f'|   NEUTRAL  | {neu:>8} |')
    print(f'+------------+----------+')
    print()
    print(f'Time: {timer.elapsed()}')

# Извлечение рецензий из КиноПоиска

In [ ]:
reviews = []

In [ ]:
import time
from IPython.display import clear_output

In [ ]:
t.start()

TOTAL_I = 10000
START_I = 10000

new_pos = 0
new_neg = 0
new_neu = 0

request_count = 0
for i in range(START_I, START_I + TOTAL_I):
    page_count, page_limit = 1, 0
    while True:
        response = get_reviews(i, page_count)
        request_count = check_wait(request_count)

        if response.status_code in (404, 402, 429):
            if response.status_code == 404:
                break
            else:
                assert False, f'\nStopped on id: {i}\n{response.status_code}\n'
            
        response = response.json()
        if not page_limit:
            page_limit = response['pagesCount']
        for review in response['reviews']:
            mark = review['reviewType']
            if mark == 'POSITIVE':
                new_pos += 1
            elif mark == 'NEGATIVE':
                new_neg += 1
            elif mark == 'NEUTRAL':
                new_neu += 1
            reviews.append((clear_text(review['reviewDescription']), mark))
        
        page_percent = round(page_count / page_limit * 100)
        update_output(new_pos, new_neg, new_neu, 
                      t, i - START_I, TOTAL_I - 1)
        
        if page_count == page_limit:
            break
        page_count += 1
        
t.stop()

# Статистика по найденным рецензиям

Поскольку для обучения имеют значение только положительные и отрицательные рецензии, нейтральные не рассматриваются

In [ ]:
def reviews_stat(reviews_list):
    all_num = len(reviews_list)
    pos_num = sum(map(lambda review: review[1] == 'POSITIVE', reviews_list))
    neg_num = sum(map(lambda review: review[1] == 'NEGATIVE', reviews_list))
    nums = [pos_num, neg_num]
    data = {'класс': ['Позитивные', 'Негативные'], 
            'число': nums, 
            'доля': [f'{round(num / all_num * 100)}%' for num in nums]}

    return pd.DataFrame(data).style.hide_index()

In [ ]:
reviews_stat(reviews)

Если по каким-либо причинам в выборке оказалось несколько идентичных рецензий, необходимо оставить только уникальные

In [ ]:
reviews_set = set(reviews)

if len(reviews) != len(reviews_set):
    reviews = list(reviews_set)

# Сохранение рецензий

In [ ]:
with open('reviews.scv', 'w') as file:
    file.write(f'review\tmark\n')
    for review, mark in reviews:
        file.write(f'{review}\t{mark}\n')